In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import random
import datetime
import math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch import Tensor
from torch.nn import Linear
import datetime

In [2]:
df = pd.read_csv('20-10_all.csv')
df.head()

,Unnamed: 0,dateTime,deviceId,uid,pm1_0,pm2_5,pm10,lat,long,pressure,temperature,humidity
0,0,2020-10-20 00:00:00,00000000078e6811,79a66147-d42f-438c-ada4-b872ba473e2c,79,126,141,28.547970,77.276649,982.329219,33.10,25.739258
1,1,2020-10-20 00:00:01,0000000024568afd,3b9159da-f4a6-4576-a2b9-b973f61f9cce,79,124,133,28.562967,77.250458,981.809687,31.70,26.247070
2,2,2020-10-20 00:00:02,00000000980cc94b,7f7f9899-b0da-466a-98ef-815ac942ecb7,82,131,147,28.579346,77.231483,983.689062,28.03,35.611328
3,3,2020-10-20 00:00:03,00000000812d59d2,dc9e628f-00e8-4b01-913a-1895542dbdbb,77,125,133,28.579374,77.230804,980.423125,23.40,46.635742
4,4,2020-10-20 00:00:03,00000000078e6811,0076dadc-09ff-455c-adf3-97116261faeb,77,125,137,28.548201,77.276474,982.376562,33.10,25.729492


In [3]:
#type casting
df.pm1_0 = df.pm1_0.astype(float)
df.pm2_5 = df.pm2_5.astype(float)
df.pm10 = df.pm10.astype(float)
df.lat = round(round(5*df.lat.astype(float),2)/5.0,3)
df.long= round(round(5*df.long.astype(float),2)/5.0,3)
df.pressure = df.pressure.astype(float)
df.temperature = df.temperature.astype(float)
df.humidity = df.humidity.astype(float)
df.dateTime = pd.to_datetime(df.dateTime)
df.head()

,Unnamed: 0,dateTime,deviceId,uid,pm1_0,pm2_5,pm10,lat,long,pressure,temperature,humidity
0,0,2020-10-20 00:00:00,00000000078e6811,79a66147-d42f-438c-ada4-b872ba473e2c,79.0,126.0,141.0,28.548,77.276,982.329219,33.10,25.739258
1,1,2020-10-20 00:00:01,0000000024568afd,3b9159da-f4a6-4576-a2b9-b973f61f9cce,79.0,124.0,133.0,28.562,77.250,981.809687,31.70,26.247070
2,2,2020-10-20 00:00:02,00000000980cc94b,7f7f9899-b0da-466a-98ef-815ac942ecb7,82.0,131.0,147.0,28.580,77.232,983.689062,28.03,35.611328
3,3,2020-10-20 00:00:03,00000000812d59d2,dc9e628f-00e8-4b01-913a-1895542dbdbb,77.0,125.0,133.0,28.580,77.230,980.423125,23.40,46.635742
4,4,2020-10-20 00:00:03,00000000078e6811,0076dadc-09ff-455c-adf3-97116261faeb,77.0,125.0,137.0,28.548,77.276,982.376562,33.10,25.729492


In [4]:
print(len(df))
# Ensuring Delhi region and removing outliers from data
df = df[(df.lat.astype(int) == 28) &(df.long.astype(int) == 77)]
df = df[(df.pm1_0<=1500) & (df.pm2_5<=1500) & (df.pm10<=1500) & (df.pm1_0>=20) & (df.pm2_5>=30) & (df.pm10>=30)]
df = df[(df.humidity<=60)&(df.humidity>=7)]
print(len(df))

97358
86892


In [5]:
df.dateTime = df.dateTime.dt.round('15min')

In [6]:
df.dateTime.head()

0   2020-10-20
1   2020-10-20
2   2020-10-20
3   2020-10-20
4   2020-10-20
Name: dateTime, dtype: datetime64[ns]

In [7]:
# rounding @15min
df.dateTime = df.dateTime.dt.round('15min')

# only PM10
dfHour = df[(df['dateTime'].dt.day == 20)][['dateTime','lat','long','pm1_0']]

In [8]:
dfHour.head()

,dateTime,lat,long,pm1_0
0,2020-10-20,28.548,77.276,79.0
1,2020-10-20,28.562,77.250,79.0
2,2020-10-20,28.580,77.232,82.0
3,2020-10-20,28.580,77.230,77.0
4,2020-10-20,28.548,77.276,77.0


In [9]:
hr_start, hr_end = 240, 300
n_servers = 3
epsilon = 1e-9
test_fraction = 1/20
valid_fraction = 0
n_epochs = 2000
n_lat_grid_lines = 100
n_long_grid_lines = 100
optimizer = "Adam"

In [10]:

lat_range = {'min': dfHour.lat.min(), 'max': dfHour.lat.max()}
long_range = {'min': dfHour.long.min(), 'max': dfHour.long.max()}

# getting grids out of location coordinates
dfHour['lat_grid'] = dfHour.apply(lambda row: int(n_lat_grid_lines*(row.lat-lat_range['min'])/(lat_range['max']-lat_range['min'])), axis=1 )
dfHour['long_grid'] = dfHour.apply(lambda row: int(n_long_grid_lines*(row.long-long_range['min'])/(long_range['max']-long_range['min'])), axis=1 )
dfHour['lat_grid'] = dfHour['lat_grid'].astype(float).astype(int)
dfHour['long_grid'] = dfHour['long_grid'].astype(float).astype(int)
del dfHour['lat']
del dfHour['long']

# converting time to minutes and selecting a one hour slot
dfHour.dateTime = dfHour.dateTime.dt.hour*60 + dfHour.dateTime.dt.minute
dfHour = dfHour[(dfHour.dateTime>=hr_start) & (dfHour.dateTime<=hr_end)] 

del dfHour['dateTime']

In [11]:
dfHour.head()

,pm1_0,lat_grid,long_grid
16440,124.0,1,97
16442,128.0,2,97
16443,92.0,48,57
16444,113.0,2,97
16445,92.0,48,57


In [12]:
# initializing servers (for experimentation)
dfHour['server'] = np.random.randint(0, 3, len(dfHour))
dfHour = dfHour.reset_index(drop=True)

In [13]:
dfHour.head()

,pm1_0,lat_grid,long_grid,server
0,124.0,1,97,0
1,128.0,2,97,1
2,92.0,48,57,2
3,113.0,2,97,2
4,92.0,48,57,2


In [14]:
# initializing servers (for experimentation)
dfHour['server'] = np.random.randint(0, 3, len(dfHour))
dfHour = dfHour.reset_index(drop=True)

# assigning labels to each grid as explained in research paper and report
grid = {}
count = 0
count2 = 0
for i,row in dfHour.iterrows():
#     print(f"{int(row.lat_grid)} {int(row.long_grid)}")
    if(not f"{int(row.lat_grid)} {int(row.long_grid)}" in grid):
        grid[f"{int(row.lat_grid)} {int(row.long_grid)}"] = count
        count += 1
        count2 += 1
# print(grid)
#     if(not f"{int(row.lat_grid + 1)} {int(row.long_grid)}" in grid):
#         grid[f"{int(row.lat_grid + 1)} {int(row.long_grid)}"] = count
#         count += 1
#     if(not f"{int(row.lat_grid)} {int(row.long_grid + 1)}" in grid):
#         grid[f"{int(row.lat_grid)} {int(row.long_grid + 1)}"] = count
#         count += 1
#     if(not f"{int(row.lat_grid - 1)} {int(row.long_grid)}" in grid):
#         grid[f"{int(row.lat_grid - 1)} {int(row.long_grid)}"] = count
#         count += 1
#     if(not f"{int(row.lat_grid)} {int(row.long_grid - 1)}" in grid):
#         grid[f"{int(row.lat_grid)} {int(row.long_grid - 1)}"] = count
#         count += 1

print(count)
print(count2)

adj = np.zeros((count, count))         # adjacency matrix of grids
aqi = np.zeros((n_servers, count))     
mask = np.zeros((n_servers, count))    

for i,row in dfHour.iterrows():
    
    mask_ser_grid = mask[int(row.server)][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]]
    aqi[int(row.server)][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]] *= mask_ser_grid/(mask_ser_grid+1)
    aqi[int(row.server)][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]] += row.pm1_0/(mask_ser_grid+1)
    mask[int(row.server)][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]] += 1
    
    # adding out-edges between neighbours because current grid contains data
    if(f"{int(row.lat_grid + 1)} {int(row.long_grid)}" in grid):
        adj[grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid + 1)} {int(row.long_grid)}"]] = 1
    if(f"{int(row.lat_grid)} {int(row.long_grid + 1)}" in grid):
        adj[grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid + 1)}"]] = 1
    if(f"{int(row.lat_grid - 1)} {int(row.long_grid)}" in grid):
        adj[grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid - 1)} {int(row.long_grid)}"]] = 1
    if(f"{int(row.lat_grid)} {int(row.long_grid - 1)}" in grid):
        adj[grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid - 1)}"]] = 1
    
    # adding in-edges between neighbours because current grid, TODO/ASK why we need to add in-edges : 
    # because it is an undirected graph
    if(f"{int(row.lat_grid + 1)} {int(row.long_grid)}" in grid):
        adj[grid[f"{int(row.lat_grid + 1)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]] = 1
    if(f"{int(row.lat_grid)} {int(row.long_grid + 1)}" in grid):
        adj[grid[f"{int(row.lat_grid)} {int(row.long_grid + 1)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]] = 1
    if(f"{int(row.lat_grid - 1)} {int(row.long_grid)}" in grid):
        adj[grid[f"{int(row.lat_grid - 1)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]] = 1
    if(f"{int(row.lat_grid)} {int(row.long_grid - 1)}" in grid):
        adj[grid[f"{int(row.lat_grid)} {int(row.long_grid - 1)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]] = 1
    
#     adj[grid[f"{int(row.lat_grid + 1)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid + 1)} {int(row.long_grid)}"]] = 1
#     adj[grid[f"{int(row.lat_grid)} {int(row.long_grid + 1)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid + 1)}"]] = 1
#     adj[grid[f"{int(row.lat_grid - 1)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid - 1)} {int(row.long_grid)}"]] = 1
#     adj[grid[f"{int(row.lat_grid)} {int(row.long_grid - 1)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid - 1)}"]] = 1
    adj[grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]][grid[f"{int(row.lat_grid)} {int(row.long_grid)}"]] = 1

# print(adj)
print(mask)
aqi_mean = (aqi*mask).sum(axis=0)/(mask.sum(axis=0) + epsilon)   # at each location from multiple servers
mask_mean = (mask).sum(axis=0)
mask_mean[mask_mean > 0] = 1
print(mask_mean)

175
175
[[ 56. 323. 214.  16.   6.   0.  14.   1.   9.   2.   1.   6.   3.   2.
    4.   0.   3.   1.   3.   1.   5.   3.   0.   3.   2.   0.  10.  22.
    3.   8.   4.   9.   1.   3.   5.   1.   1.  22.   0.  22.   2.   7.
    5.   4.  14.   2.   4.   8.   1.   2.   3.   1.   8.   4.   4.   6.
    4.   3.   8.   6.   3.   0.   0.   0.   7.   2.   4.   0.   1.   4.
    8.   2.   1.  21.   3.   4.   2.  34.   1.   4.   7.   1.   2.   0.
    2.   1.   5.   4.   2.   3.   9.   3. 114.   0.   1.   6.   1.   0.
    7.   0.   1.   0.   1.   1.   6.   1.   0.   2.   0.   2.   1.   2.
    1.   2.   1.   1.   1.   4.   0.  10.   0.   1.   6.   3.   4.   0.
    2.   2.   2.   5.   7.   2.   2.   4.   0.   0.   6.   0.   2.   0.
   28.   3.   2.   3.   1.   0.   0.   3.   1.   1.   1.   0.   3.   2.
    1.   1.   0.   3.   5.   3.   5.   3.   3.   2.   3.   3.  10.   2.
   11.   1.   0.   3.   4.   1.   1.]
 [ 57. 315. 256.  20.   7.   4.   7.   0.   5.   6.   0.   6.   7.   2.
    2.   0.   3.  

In [19]:
# getting 1/20 locations for test
test_indices = random.sample(range(aqi_mean.shape[0]), int(aqi_mean.shape[0] * test_fraction))

# making grids for test to value 0
aqi_mean_train = aqi_mean * 1
aqi_mean_train[test_indices] = 0

mask_mean_train = mask_mean * 1
mask_mean_train[test_indices] = 0

adj_train = np.array(adj)
adj_train[:, test_indices] = 0

print(mask_mean_train.shape)

# adj[:, test_indices] = 0

(175,)


In [16]:
train_data = {
    'adj': torch.FloatTensor(adj_train),
    'aqi': torch.FloatTensor(aqi_mean_train).view(-1,1),
    'mask': torch.FloatTensor(mask_mean_train)
}

data = {
    'adj': torch.FloatTensor(adj),
    'aqi': torch.FloatTensor(aqi_mean).view(-1,1),
    'mask': torch.FloatTensor(mask_mean)
}

In [18]:
print(train_data)

{'adj': tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]]), 'aqi': tensor([[116.7012],
        [111.6151],
        [ 85.8906],
        [ 81.0615],
        [117.7895],
        [126.5000],
        [121.0000],
        [ 34.0000],
        [  0.0000],
        [120.0000],
        [115.6667],
        [114.9474],
        [112.8824],
        [120.2500],
        [116.7273],
        [117.0000],
        [115.1667],
        [118.4286],
        [113.0000],
        [114.5000],
        [115.2727],
        [111.3636],
        [108.2500],
        [109.7000],
        [121.1538],
        [132.3333],
        [127.0357],
        [118.2143],
        [122.1250],
        [120.8261],
        [114.2727],
        [123.3500],
        [131.0000],
        [125.4286],
        [122.3846],
        [  0.0000],
        [1

In [21]:
x, adj = torch.cat(train_data['aqi'],train_data['mask'].view(-1,1),1), train_data['adj']

TypeError: cat() received an invalid combination of arguments - got (Tensor, Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [22]:
class GCNConv(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GCNConv, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        # initialzes weights with standard deviation  = 1/sqrt(weight.size())
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj, mask):
        D = 1 / (0.0000001 + adj.sum(axis=1).unsqueeze(-1))
#         D = 1 / (1 + torch.mm(adj, mask.unsqueeze(-1)))
#         print(D)
        support = torch.mm(input, self.weight)
        output = D * torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
    
# TODO/ASK: too many parameters
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1Mean = GCNConv(2,8)
        self.conv2Mean = GCNConv(8,4)
#         self.conv3Mean = GCNConv(4,4)
        self.lrelu = nn.LeakyReLU(0.1)
        self.lin3 = nn.Linear(4, 1)

    def forward(self,data):
#         x, adj = data['aqi'], data['adj']
        x, adj = torch.cat((data['aqi'],data['mask'].view(-1,1)),1), data['adj']
#         x = F.relu(self.conv1Mean(x, adj, data['mask']))
        x = self.lrelu(self.conv1Mean(x, adj, data['mask']))
        
#         x = F.relu(self.conv2Mean(x, adj, data['mask']))
        x = self.lrelu(self.conv2Mean(x, adj, data['mask']))
    
#         x = F.relu(self.conv3Mean(x, adj, data['mask']))
        x = self.lin3(x)
        return torch.squeeze(x)
    
model = Net()
if (optimizer == "SGD"):
#     opt = torch.optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.01, momentum = 0.1, nesterov=True)
    opt = torch.optim.SGD(model.parameters(), lr=0.0001) 
else:
    opt = torch.optim.Adam(model.parameters(), lr = 0.01)

In [23]:
x, adj = torch.cat((train_data['aqi'],train_data['mask'].view(-1,1)),1), train_data['adj']

tensor([[116.7012,   1.0000],
        [111.6151,   1.0000],
        [ 85.8906,   1.0000],
        [ 81.0615,   1.0000],
        [117.7895,   1.0000],
        [126.5000,   1.0000],
        [121.0000,   1.0000],
        [ 34.0000,   1.0000],
        [  0.0000,   0.0000],
        [120.0000,   1.0000],
        [115.6667,   1.0000],
        [114.9474,   1.0000],
        [112.8824,   1.0000],
        [120.2500,   1.0000],
        [116.7273,   1.0000],
        [117.0000,   1.0000],
        [115.1667,   1.0000],
        [118.4286,   1.0000],
        [113.0000,   1.0000],
        [114.5000,   1.0000],
        [115.2727,   1.0000],
        [111.3636,   1.0000],
        [108.2500,   1.0000],
        [109.7000,   1.0000],
        [121.1538,   1.0000],
        [132.3333,   1.0000],
        [127.0357,   1.0000],
        [118.2143,   1.0000],
        [122.1250,   1.0000],
        [120.8261,   1.0000],
        [114.2727,   1.0000],
        [123.3500,   1.0000],
        [131.0000,   1.0000],
        [1

In [27]:
adj

tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])